In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT', 'SQ', 'STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-10a', mod, True, loss=lossfn)[0]

In [12]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
58,SPOT,RROnly,1642.45,1595.89,3238.34,76.048686,0.936170,160.53,189.119995,0.18,9.007092
59,SQ,RROnly,1454.32,1313.40,2767.72,42.036880,0.936170,43.83,49.599998,0.13,8.870704
20,ETSY,RROnly,1393.41,1429.52,2822.93,36.434573,0.925532,62.72,71.839996,0.15,8.758865
29,INTA,RROnly,141.44,109.88,251.32,14.464197,0.921569,36.10,29.969999,-0.17,8.754325
3,AMBA,RROnly,1098.39,985.75,2084.14,40.325995,0.936170,53.40,48.259998,-0.10,8.723404
46,PANW,RROnly,1211.50,734.36,1945.86,47.989190,0.914894,246.69,196.229996,-0.20,8.723404
12,DASH,RROnly,475.43,1015.29,1490.72,46.704974,0.915385,74.51,87.510002,0.17,8.656109
60,STEM,RROnly,192.04,268.07,460.11,4.524527,0.927536,3.94,4.350000,0.10,8.550725
43,OLED,RROnly,984.22,1177.17,2161.39,92.594497,0.920213,158.78,178.559998,0.12,8.537234
54,RBLX,RROnly,320.98,636.26,957.24,25.263613,0.957627,30.46,29.389999,-0.04,8.516949


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-7a', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
2,ARKW,RROnly,805.91,786.66,1592.57,38.158114,0.984043,57.96,65.870003,0.14,9.726444
0,ARKF,RROnly,261.97,224.44,486.41,14.147363,0.962766,21.06,23.840000,0.13,9.341244
23,SMOG,RROnly,873.66,661.55,1535.21,51.864062,0.957447,118.43,133.539993,0.13,9.247136
30,XITK,RROnly,1008.97,827.19,1836.16,86.468085,0.952128,128.30,141.190002,0.10,9.042553
10,IPO,RROnly,351.14,314.69,665.83,22.307335,0.946809,32.46,35.830002,0.10,8.936170
43,XNTK,RROnly,832.40,391.70,1224.10,64.272084,0.968085,138.26,143.940002,0.04,8.882979
20,ONLN,RROnly,281.45,384.87,666.32,27.105897,0.930851,34.85,36.840000,0.06,8.156028
7,FPX,RROnly,472.74,544.47,1017.21,55.536532,0.946809,87.01,90.330002,0.04,8.138298
11,IXC,RROnly,133.59,126.75,260.34,11.540287,0.898936,40.37,36.320000,-0.10,7.978723
17,MGK,RROnly,1040.64,666.63,1707.26,111.184990,0.952128,232.96,226.830002,-0.03,7.925532


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
             'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'RAD', 'REAL',  'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4a', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
31,MTTR,RROnly,87.65,101.53,189.17,2.689209,0.916667,2.35,2.930000,0.25,5.058824
44,STEM,RROnly,160.70,196.65,357.35,4.544574,0.911111,4.82,5.760000,0.19,4.891641
1,APRN,RROnly,1411.97,1394.92,2806.89,7.790006,0.877660,6.12,8.560000,0.40,4.856852
33,NSTG,RROnly,374.46,461.83,836.29,3.059434,0.861702,1.70,2.570000,0.51,4.797664
22,GTE,RROnly,101.03,139.79,240.82,2.181307,0.904255,6.57,5.360000,-0.18,4.787234
15,DMTK,RROnly,298.63,326.06,624.69,2.680354,0.840426,1.81,3.170000,0.75,4.730914
47,YEXT,RROnly,72.86,112.67,185.53,4.741613,0.829787,6.31,9.540000,0.51,4.547351
21,GOVX,RROnly,28.79,41.56,70.35,0.624102,0.868613,0.54,0.660000,0.22,4.512276
12,CLOV,RROnly,56.48,80.07,136.56,1.022691,0.848684,1.13,1.470000,0.31,4.504145
14,DLO,RROnly,121.56,232.08,353.64,11.205691,0.912621,20.30,22.379999,0.10,4.494575


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-9b', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,BA,RROnly,1689.83,1910.34,3600.17,99.861119,0.882979,208.11,244.000000,0.17,5.618630
36,UNH,RROnly,2173.63,1095.84,3269.47,191.452462,0.904255,486.70,434.630005,-0.11,5.588560
26,NKE,RROnly,611.59,634.27,1245.86,63.144150,0.904255,96.26,105.959999,0.10,5.501520
8,BRK-B,RROnly,1238.40,782.28,2020.68,165.411500,0.925532,367.86,346.410004,-0.06,5.369807
18,FDX,RROnly,1603.09,1124.37,2727.46,89.652237,0.914894,254.08,235.289993,-0.07,5.319149
31,QCOM,RROnly,902.29,816.88,1719.17,59.348742,0.893617,113.14,101.709999,-0.10,5.319149
21,IBM,RROnly,545.38,488.18,1033.56,77.640117,0.898936,145.99,132.619995,-0.09,5.298041
2,ADBE,RROnly,3399.28,2374.43,5773.71,269.761160,0.898936,528.89,567.510010,0.07,4.977204
10,CDNS,RROnly,1000.50,518.44,1518.94,57.468848,0.904255,234.08,218.899994,-0.06,4.863222
15,DHR,RROnly,1184.86,956.14,2141.00,124.142693,0.882979,255.25,233.960007,-0.08,4.844225


In [ ]:
findata.EPOCHS=200
tickers5 = ['AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-8b', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
43,ONON,RROnly,136.82,132.79,269.61,16.755016,0.932584,28.08,35.299999,0.26,6.142734
5,ADYEY,RROnly,158.31,165.64,323.95,6.895624,0.904255,6.84,9.630000,0.41,5.991919
33,MGA,RROnly,411.03,384.22,795.25,23.180247,0.920213,53.33,61.950001,0.16,5.575608
0,AEXAF,RROnly,198.15,467.43,665.58,8.711679,0.819149,7.15,13.620000,0.90,5.449173
54,SE,RROnly,1427.40,1489.73,2917.13,32.842804,0.893617,35.96,42.820000,0.19,5.263158
55,SIEGY,RROnly,393.04,410.03,803.07,31.150417,0.946809,70.64,75.320000,0.07,5.243161
23,FANUY,RROnly,112.44,100.74,213.18,11.675101,0.914894,13.38,14.920000,0.12,5.116515
57,TEF,RROnly,20.79,21.45,42.24,2.651354,0.914894,4.28,3.850000,-0.10,4.832827
65,VWAPY,RROnly,80.12,91.32,171.44,6.926266,0.898936,12.08,10.680000,-0.12,4.736575
59,THQQF,RROnly,137.58,211.14,348.71,2.586903,0.819149,2.25,2.980000,0.32,4.720745


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-9b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
8,EWC,RROnly,112.93,105.84,218.77,16.802443,0.920213,33.95,36.750000,0.08,8.204787
1,EWD,RROnly,181.14,142.77,323.91,20.022881,0.898936,32.49,36.439999,0.12,8.147163
3,FLN,RROnly,72.36,110.67,183.03,8.887277,0.914894,18.63,19.540001,0.05,7.446809
16,KEMQ,RROnly,94.59,131.55,226.13,11.979621,0.829787,14.10,16.719999,0.19,7.402016
4,FM,RROnly,90.97,83.56,174.53,18.162559,0.888298,26.35,27.870001,0.06,7.021277
0,ARGT,RROnly,189.94,135.91,325.86,14.386260,0.930851,42.91,41.810001,-0.03,7.003546
19,XCEM,RROnly,71.31,91.70,163.01,16.202963,0.909574,27.63,26.549999,-0.04,6.835106
7,EWI,RROnly,113.49,105.93,219.42,16.506191,0.872340,31.02,32.560001,0.05,6.170213
13,EWZ,RROnly,147.98,227.46,375.44,16.721452,0.936170,31.17,31.690001,0.02,6.170213
10,EWP,RROnly,127.67,104.40,232.08,16.638516,0.909574,27.85,28.700001,0.03,6.081560


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ALB', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-8b', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
64,TGT,RROnly,814.04,991.01,1805.04,87.177983,0.952128,110.57,119.029999,0.08,8.922872
67,UAL,RROnly,324.33,608.84,933.17,20.796685,0.925532,42.30,50.060001,0.18,8.841608
8,BGFV,RROnly,173.90,112.50,286.40,1.009523,0.904255,7.01,9.790000,0.40,8.803191
41,KMB,RROnly,493.75,476.25,970.00,100.432847,0.962766,120.85,125.169998,0.04,8.696809
17,CG,RROnly,210.49,205.51,416.01,15.798222,0.914894,30.16,35.470001,0.18,8.676123
3,AQN,RROnly,44.80,57.14,101.94,6.066132,0.914894,5.92,6.950000,0.17,8.648310
35,IP,RROnly,187.34,152.85,340.20,21.783540,0.920213,35.47,40.590000,0.14,8.632219
63,TCN,RROnly,20.12,39.35,59.46,7.202173,0.931034,7.40,8.160000,0.10,8.620690
57,SAFE,RROnly,561.07,554.24,1115.31,19.019964,0.898936,17.80,22.230000,0.25,8.585106
36,IPI,RROnly,409.31,567.68,976.99,8.349134,0.914894,25.16,28.570000,0.14,8.541033


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-9b', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,TLT,RROnly,280.14,350.09,630.22,89.821823,0.888298,88.69,94.470001,0.07,7.287234
7,UUP,RROnly,48.75,25.53,74.28,23.992486,0.930851,29.73,29.240000,-0.02,5.851064
3,SLV,RROnly,121.56,110.78,232.34,11.535923,0.904255,20.34,19.969999,-0.02,4.255319
1,ISHG,RROnly,86.07,119.54,205.61,63.663234,0.941489,67.58,68.089996,0.01,3.563830
0,GLD,RROnly,503.74,374.87,878.61,138.373135,0.936170,171.45,173.360001,0.01,2.978723
4,SOYB,RROnly,79.83,38.03,117.86,13.664672,0.851064,27.07,27.590000,0.02,1.063830
2,PDBC,RROnly,55.98,33.41,89.39,6.626020,0.898936,14.95,14.990000,0.00,0.000000
5,TIP,RROnly,112.32,90.78,203.10,95.390300,0.835106,103.72,104.769997,0.01,0.000000
8,VTIP,RROnly,30.72,19.22,49.94,41.111158,0.898936,47.28,47.380001,0.00,0.000000
9,WEAT,RROnly,31.02,33.32,64.34,5.018437,0.829787,5.57,5.590000,0.00,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-9a', mod9, False)[0]

In [8]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,MATIC-USD,RROnly,15.02,18.44,33.45,0.100220,0.845745,0.57,0.450000,-0.20,5.269757
4,ETH-USD,RROnly,20140.61,31764.82,51905.43,1124.038393,0.856383,1649.82,1457.839966,-0.12,4.920213
5,LTC-USD,RROnly,1447.57,1608.11,3055.68,51.650611,0.819149,65.73,57.549999,-0.12,4.343972
3,DOGE-USD,RROnly,1.85,1.76,3.61,0.022139,0.734043,0.06,0.050000,-0.26,4.220248
1,BTC-USD,RROnly,271802.48,213163.55,484966.03,17380.985081,0.819149,28066.96,25042.580078,-0.11,4.206963
2,DOT-USD,RROnly,230.97,259.58,490.56,5.190831,0.792553,4.08,3.630000,-0.11,3.775214
0,ADA-USD,RROnly,8.80,13.96,22.77,0.304852,0.739362,0.27,0.230000,-0.13,3.276245
7,XRP-USD,RROnly,8.15,7.86,16.01,0.314806,0.734043,0.53,0.460000,-0.13,3.197335


In [14]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
ipos-10a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,094 bytes  received 35 bytes  6,442,258.00 bytes/sec
total size is 179,896,192  speedup is 55.85
/content/drive/MyDrive/colab/results/val-6b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-6b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-6b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-6b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/cry-6b-test-adjc